Problem statement
- Design a data structure that supports:
1. **Insert(timestamp, value)**: add a new measurement
2. **QueryMedian(N)**: return the median of the last N measurements
- We then discuss the time complexity

Median
- to maintain the median of a dynamic set efficiently, we keep numbers split into two heaps
- max-heap: lower half
- min-heap: upper half
- the median is top of max-heap (if odd) or average of top elements (if even)
- this allows **$O(\log n)$** insertions ans **$O(1)$** median

Sliding window
- we need to remove the element that exits the last-N window
- *Lazy deletion*: we keep two heaps and a dictionary for elements that must be removed later
- when a heap top element is marked for deletion, we pop and continue

Complexity
- insert $\rightarrow$ $O(\log N)$
- delete old element $\rightarrow$ $O(\log N)$
- query median $\rightarrow$ $O(1)$

Data structure
- two heaps: `lo` (max-heap) and `hi` (min-heap)
- a queue to track timestamps
- a dict for lazy deletion

In [ ]:
import heapq
from collections import defaultdict, deque

class SlidingMedian:
    def __init__(self):
        self.lo = [] # max heap (store negatives)
        self.hi = [] # min heap
        self.delayed = defaultdict(int)
        self.window = deque()
        self.lo_size = 0
        self.hi_size = 0
    
    def insert(self, value):
        # insert into max heap or min heap based on value
        if not self.lo or value <= -self.lo[0]:
            heapq.heappush(self.lo, -value)
            self.lo_size += 1
        else:
            heapq.heappush(self.hi, value)
            self.hi_size += 1
        
        self.window.append(value)
        self._rebalance()
    
    def _rebalance(self):
        # ensure size(lo) >= size(hi)
        if self.lo_size > self.hi_size + 1: # lo --> hi
            val = -heapq.heappop(self.lo)
            heapq.heappush(self.hi, val)
            self.lo_size -= 1
            self.hi_size += 1
        elif self.hi_size > self.lo_size: # hi --> lo
            val = heapq.heappop(self.hi)
            heapq.heappush(self.lo, -val)
            self.hi_size -= 1
            self.lo_size += 1

    def _prune(self, heap):
        # removes elements that should be deleted
        while heap:
            value = -heap[0] if heap is self.lo else heap[0]
            if self.delayed[value] > 0:
                heapq.heappop(heap)
                self.delayed[value] -= 1
            else:
                break

    def remove_oldest(self):
        old = self.window.popleft()
        self.delayed[old] += 1

        # decrement size but don't remove immediately (lazy)
        if old <= -self.lo[0]:
            self.lo_size -= 1
        else:
            self.hi_size -= 1
        
        # clean tops if necessary
        self._prune(self.lo)
        self._prune(self.hi)
        self._rebalance()
    
    def median(self):
        # if heaps are balanced or lo has 1 extra
        if (self.lo_size + self.hi_size) % 2 == 1: # higher of the lower group
            return float(-self.lo[0])
        else:
            return ( -self.lo[0] + self.hi[0] ) / 2.0 # mean 
    
    def query_last_n(self, N):
        # remove older elements until window has N items
        while len(self.window) > N:
            self.remove_oldest()
        
        return self.median()

Insert(value)
- put value in the appropriate heap
- rebalance the heaps
- append to queue


QueryMedian(N)
- if window > N $\rightarrow$ remove oldest
- median is top of heaps


Lazy delation
- we only clean invalid elements when they reach heap top

Test

In [4]:
def print_state(sm):
    print("  lo (max-heap):", [-x for x in sm.lo])  # invert for readability
    print("  hi (min-heap):", sm.hi)
    print("  window:", list(sm.window))
    print("")

sm = SlidingMedian()

# test stream of measurements
stream = [3, 5, 2, 8, 12, 3, 15, 7]
N = 5 # sliding window size


print("=== Inserting values ===\n")
for value in stream:
    print(f"Inserting value {value}")
    sm.insert(value)
    print_state(sm)

    if len(sm.window) >= N:
        med = sm.query_last_n(N)
        print(f"Median of last {N} values = {med}")
        print("-" * 40)


=== Inserting values ===

Inserting value 3
  lo (max-heap): [3]
  hi (min-heap): []
  window: [3]

Inserting value 5
  lo (max-heap): [3]
  hi (min-heap): [5]
  window: [3, 5]

Inserting value 2
  lo (max-heap): [3, 2]
  hi (min-heap): [5]
  window: [3, 5, 2]

Inserting value 8
  lo (max-heap): [3, 2]
  hi (min-heap): [5, 8]
  window: [3, 5, 2, 8]

Inserting value 12
  lo (max-heap): [5, 2, 3]
  hi (min-heap): [8, 12]
  window: [3, 5, 2, 8, 12]

Median of last 5 values = 5.0
----------------------------------------
Inserting value 3
  lo (max-heap): [3, 3, 2]
  hi (min-heap): [5, 12, 8]
  window: [3, 5, 2, 8, 12, 3]

Median of last 5 values = 5.0
----------------------------------------
Inserting value 15
  lo (max-heap): [5, 2, 3]
  hi (min-heap): [8, 12, 15]
  window: [5, 2, 8, 12, 3, 15]

Median of last 5 values = 8.0
----------------------------------------
Inserting value 7
  lo (max-heap): [7, 2, 3]
  hi (min-heap): [8, 15, 12]
  window: [2, 8, 12, 3, 15, 7]

Median of last 5 va